# Cognitive Decision Graph (CDG) Demo

This notebook demonstrates the CDG data structure for analyzing Python programs at a cognitive/semantic level.


In [1]:
from cdg import (
    CDGBuilder, build_cdg, compare_programs, print_cdg,
    Interval, Region,
    InputNode, ControlNode, BranchNode, ActionNode, BodyNode
)


## Example 1: Basic Astronaut Height Checker

This is the first example program with if/elif/else structure.


In [3]:
code_1 = '''
def main():
    height = float(input("Enter your height in meters: "))
    if height < 1.6:
        print("Below minimum astronaut height")
    elif 1.6 < height < 1.9:
        print("Correct height to be an astronaut")
    else:
        print("Above maximum astronaut height")
'''

print("=== Program 1 CDG ===")
print_cdg(code_1)


=== Program 1 CDG ===
BodyNode(main)
  InputNode(type=float, prompt="Enter your height in meters: ")
  ControlNode(type=if, branches=3)
    BranchNode(if/elif, region=(-∞, 1.6))
      ActionNode(print, msg="Below minimum astronaut height")
    BranchNode(if/elif, region=(1.6, 1.9))
      ActionNode(print, msg="Correct height to be an astronaut")
    BranchNode(else, region={1.6} ∪ [1.9, ∞))
      ActionNode(print, msg="Above maximum astronaut height")


## Example 2: While Loop Variant

Same logic but wrapped in a `while True` loop with different condition ordering.


In [6]:
code_2 = '''
def main():
    height = float(input("Enter your height in meters: "))
    if height > 1.6 and height < 1.9:
        print("Correct height to be an astronaut")
    elif height <= 1.6:
        print("Below minimum astronaut height")
    elif height >= 1.9:
        print("Above maximum astronaut height")
'''

print("=== Program 2 CDG ===")
print_cdg(code_2)


=== Program 2 CDG ===
BodyNode(main)
  InputNode(type=float, prompt="Enter your height in meters: ")
  ControlNode(type=if, branches=3)
    BranchNode(if/elif, region=(1.6, 1.9))
      ActionNode(print, msg="Correct height to be an astronaut")
    BranchNode(if/elif, region=(-∞, 1.6])
      ActionNode(print, msg="Below minimum astronaut height")
    BranchNode(if/elif, region=[1.9, ∞))
      ActionNode(print, msg="Above maximum astronaut height")


## Compare the Two Programs


In [5]:
similarity = compare_programs(code_1, code_2)

print("\n=== Similarity Analysis ===")
for metric, score in similarity.items():
    print(f"  {metric:15}: {score:.2f}")



=== Similarity Analysis ===
  structural     : 0.45
  behavioral     : 1.00
  regional       : 0.20
  input          : 1.00
  action         : 1.00
  overall        : 0.49


## Interval Arithmetic Demo

The Region class handles complex interval operations.


In [ ]:
# Create intervals
below_1_6 = Interval(float('-inf'), 1.6, False, False)  # (-inf, 1.6)
above_1_9 = Interval(1.9, float('inf'), True, False)     # [1.9, inf)

print(f"Below 1.6: {below_1_6}")
print(f"Above 1.9: {above_1_9}")

# Create regions from constraints
region_lt_1_6 = Region.from_constraint('<', 1.6)
region_gt_1_6 = Region.from_constraint('>', 1.6)
region_lt_1_9 = Region.from_constraint('<', 1.9)

print(f"\nRegion (height < 1.6): {region_lt_1_6}")
print(f"Region (height > 1.6): {region_gt_1_6}")

# Intersection (AND)
correct_height = region_gt_1_6.intersect(region_lt_1_9)
print(f"\nCorrect height (1.6 < h < 1.9): {correct_height}")

# Complement (ELSE)
not_correct = correct_height.complement()
print(f"Not correct height: {not_correct}")


## Equivalent Programs with Different Syntax

Let's compare syntactically different but semantically equivalent programs.


In [5]:
# Version A: Uses 'and' operator
code_a = '''
def main():
    h = float(input("Height: "))
    if h > 1.6 and h < 1.9:
        print("OK")
    else:
        print("Not OK")
'''

# Version B: Uses chained comparison
code_b = '''
def main():
    height = float(input("Height: "))
    if 1.6 < height < 1.9:
        print("OK")
    else:
        print("Not OK")
'''

print("=== Program A (uses 'and') ===")
print_cdg(code_a)

print("\n=== Program B (uses chained comparison) ===")
print_cdg(code_b)

print("\n=== Similarity ===")
sim = compare_programs(code_a, code_b)
for k, v in sim.items():
    print(f"  {k}: {v:.2f}")


=== Program A (uses 'and') ===
BodyNode(main)
  InputNode(type=float, prompt="Height: ")
  ControlNode(type=if, branches=2)
    BranchNode(if/elif, region=(1.6, 1.9))
      ActionNode(print, msg="OK")
    BranchNode(else, region=(-∞, 1.6] ∪ [1.9, ∞))
      ActionNode(print, msg="Not OK")

=== Program B (uses chained comparison) ===
BodyNode(main)
  InputNode(type=float, prompt="Height: ")
  ControlNode(type=if, branches=2)
    BranchNode(if/elif, region=(1.6, 1.9))
      ActionNode(print, msg="OK")
    BranchNode(else, region=(-∞, 1.6] ∪ [1.9, ∞))
      ActionNode(print, msg="Not OK")

=== Similarity ===
  structural: 1.00
  behavioral: 1.00
  regional: 1.00
  input: 1.00
  action: 1.00
  overall: 1.00


In [6]:
# Build CDG and inspect structure
builder = CDGBuilder(code_1)
cdg = builder.get_cdg()

print("Root:", cdg)
print("\nChildren:")
for child in cdg.children:
    print(f"  {child}")
    
    if isinstance(child, ControlNode):
        print(f"    Branches:")
        for branch in child.branches:
            print(f"      {branch}")
            for action in branch.children:
                print(f"        -> {action}")


Root: BodyNode(main)

Children:
  InputNode(type=float, prompt="Enter your height in meters: ")
  ControlNode(type=if, branches=3)
    Branches:
      BranchNode(if/elif, region=(-∞, 1.6))
        -> ActionNode(print, msg="Below minimum astronaut height")
      BranchNode(if/elif, region=(1.6, 1.9))
        -> ActionNode(print, msg="Correct height to be an astronaut")
      BranchNode(else, region={1.6} ∪ [1.9, ∞))
        -> ActionNode(print, msg="Above maximum astronaut height")


In [2]:
# Version A
code_a = '''
def main():
    while True:
        height = float(input("Enter your height in meters: "))

        if height > 1.6 and height < 1.9:
            print("Correct height to be an astronaut")
        elif height <= 1.6:
            print("Below minimum astronaut height")
        elif height >= 1.9:
            print("Above maximum astronaut height")
'''

# Version B
code_b = '''
def main():
    your_height = float(input("Enter your height in meters: "))

    if your_height >= 1.9:
        print("Above maximum astronaut height")
    elif your_height <= 1.6:
        print("Below minimum astronaut height")
    else:
        print("Correct height to be an astronaut")
'''

print("=== Program A (uses 'and') ===")
print_cdg(code_a)

print("\n=== Program B (uses chained comparison) ===")
print_cdg(code_b)

print("\n=== Similarity ===")
sim = compare_programs(code_a, code_b)
for k, v in sim.items():
    print(f"  {k}: {v:.2f}")


=== Program A (uses 'and') ===
BodyNode(main)
  ControlNode(type=while, branches=0)
    InputNode(type=float, prompt="Enter your height in meters: ")
    ControlNode(type=if, branches=3)
      BranchNode(if/elif, region=(1.6, 1.9))
        ActionNode(print, msg="Correct height to be an astronaut")
      BranchNode(if/elif, region=(-∞, 1.6])
        ActionNode(print, msg="Below minimum astronaut height")
      BranchNode(if/elif, region=[1.9, ∞))
        ActionNode(print, msg="Above maximum astronaut height")

=== Program B (uses chained comparison) ===
BodyNode(main)
  InputNode(type=float, prompt="Enter your height in meters: ")
  ControlNode(type=if, branches=3)
    BranchNode(if/elif, region=[1.9, ∞))
      ActionNode(print, msg="Above maximum astronaut height")
    BranchNode(if/elif, region=(-∞, 1.6])
      ActionNode(print, msg="Below minimum astronaut height")
    BranchNode(else, region=(1.6, 1.9))
      ActionNode(print, msg="Correct height to be an astronaut")

=== Similarity